In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from nodevectors import Node2Vec

#### Prepare the raw data

In [2]:
brand_ids = pd.read_csv('./clean/brand_ids.csv', low_memory=False)
account_brands = pd.read_csv('./clean/accounts-brands.csv', low_memory=False)

interest_ids = pd.read_csv('./clean/interest_ids.csv', low_memory=False)
account_interests = pd.read_csv('./clean/accounts-interests.csv', low_memory=False)

city_ids = pd.read_csv('./clean/city_ids.csv', low_memory=False)
country_ids = pd.read_csv('./clean/country_ids.csv', low_memory=False)

users = pd.read_csv('./clean/users.csv', low_memory=False)

In [3]:
brand_ids['label'] = brand_ids['Name'].apply(lambda x: f'Brand:{x}')
account_brands['Account Id'] = account_brands['Account Id'].apply(lambda x: f'Account:{x}')
account_brands['Brand Id'] = account_brands['Brand Id'].map(brand_ids.set_index('Id')['label'].to_dict())

interest_ids['label'] = interest_ids['Name'].apply(lambda x: f'Interest:{x}')
account_interests['Account Id'] = account_interests['Account Id'].apply(lambda x: f'Account:{x}')
account_interests['Interest Id'] = account_interests['Interest Id'].map(interest_ids.set_index('Id')['label'].to_dict())

# city_ids['label'] = city_ids['Name'].apply(lambda x: f'City:{x}')
# country_ids['label'] = country_ids['Name'].apply(lambda x: f'Country:{x}')

users['label'] = users['Id'].apply(lambda x: f'Account:{x}')

#### Transform

In [4]:
numeric_data = users.iloc[:, 4:29]

take_the_log = [
    'Followers', 'Following', 'Posts', 
    'Engagement', 'Estimated reach', 'Estimated impressions', 
    'Avg. likes per post', 'Avg. engagement per post',
    'Avg. comments per post', 'Avg. views per video',
    'Cost per story (MIN)', 'Cost per story (MAX)'
    ]
for column in take_the_log:
    numeric_data[column] = numeric_data[column].apply(lambda x: 0.1 if (x == 0) else x)
    numeric_data[column] = numeric_data[column].apply(np.log)

In [5]:
numeric_as_categorical = numeric_data.apply(lambda x: pd.cut(x, 3, labels=["low", "average", "high"], duplicates='drop'))

In [6]:
clean_df = pd.concat([users.iloc[:, :4], numeric_as_categorical], axis=1)
clean_df['Id'] = clean_df['Id'].apply(lambda x: f'Account:{x}')
clean_df = clean_df.rename(columns={'Country id': 'Country', 'City id': 'City'}).set_index('Id')

clean_df['City'] = clean_df['City'].map(city_ids.set_index('Id')['Name'].to_dict())
clean_df['Country'] = clean_df['Country'].map(country_ids.set_index('Id')['Name'].to_dict())

#### Create edges

In [7]:
edges = []
for column in clean_df.columns:
    series_tuples = [
        (account, f'{column}:{value}') for account, value in clean_df[column].items()
        ]
    edges.extend(series_tuples)

In [8]:
len(edges)

579712

In [9]:
edges += [i for i in account_brands.set_index('Account Id')['Brand Id'].items()]

In [10]:
len(edges)

710822

In [11]:
edges += [i for i in account_interests.set_index('Account Id')['Interest Id'].items()]

In [12]:
len(edges)

851854

In [13]:
temp = users.set_index('label').iloc[:, 29:-2]
for column in temp:
    series_tuples = [
        (account, f'{column}:True') for account, value in temp[column].items()
        ]
    edges.extend(series_tuples)

In [14]:
len(edges)

955374

#### Generate node embeddings

In [19]:
G = nx.from_pandas_edgelist(pd.DataFrame(edges, columns=['source', 'target']))

In [20]:
g2v = Node2Vec()
g2v.fit(G)

/Users/blake/miniconda3/lib/python3.8/site-packages/networkx/linalg/graphmatrix.py:187: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return adjacency_matrix(G, nodelist, dtype, weight)


AttributeError: module 'scipy.sparse' has no attribute 'coo_array'